# Multi-layer LSTM for Music Generation (with Magenta)

In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
import note_seq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dropout, Dense, Activation, Lambda, Softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from note_seq.midi_io import midi_file_to_note_sequence

note_divisions = 32
sequence_length = 100

In [2]:
def create_network(network_input, n_vocab):
    """ Define constants """
    hidden_layers = 256
    dropout = 0.4
    temp = 0.6
    
    """ Initializing model """
    model = Sequential()
    
    """ Adding LSTM Layers to Model """
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout,
                return_sequences=True
            )
        )
    )
    model.add(
        Bidirectional(
            LSTM(
                hidden_layers,
                dropout=dropout
            )
        )
    )
    
    """ Add other layers after LSTM """
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(hidden_layers // 2))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x / temp))
    model.add(Softmax())
    
    """ Define the loss function for the model """
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    
    return model
    

In [3]:
# vocab key format "flag pitch"
# flag 0 is next channel, flag 1 is new note, flag 2 is next step, flag 3 is continue note

def is_start(target, start):
    return target + (0.5 / note_divisions) > start and target - (0.5 / note_divisions) < start

def is_note_valid(target, note, instrument_infos):
    return (
        note.end_time - note.start_time >= 1 / note_divisions and
        target - (0.5 / note_divisions) < note.end_time and
        target + (0.5 / note_divisions) > note.start_time and
        instrument_infos[note.instrument].name != 'no'
    )

def get_vocab():
    vocab = set(['0 0', '2 0'])
    for file in os.listdir('samples/small'):
        data = midi_file_to_note_sequence('samples/small/' + file)
        total_time = data.time_signatures[1].time
        iteration = 0
        while iteration / note_divisions < total_time:
            continue_notes = [
                note for note in data.notes
                if is_note_valid(iteration / note_divisions, note, data.instrument_infos) and
                not is_start(iteration / note_divisions, note.start_time)
            ]
            new_notes = [
                note for note in data.notes
                if is_note_valid(iteration / note_divisions, note, data.instrument_infos) and
                is_start(iteration / note_divisions, note.start_time)
            ]
            for note in continue_notes:
                vocab.add(str(3) + ' ' + str(note.pitch))
            for note in new_notes:
                vocab.add(str(1) + ' ' + str(note.pitch))
            iteration += 1
    vocab = list(vocab)
    vocab.sort()
    pickle.dump(vocab, open('vocab.p', 'wb'))
    return vocab

def get_notes(vocab):
    vocab_dict = {k: v for v, k in enumerate(vocab)}
    notes = []
    for file in os.listdir('samples/small'):
        data = midi_file_to_note_sequence('samples/small/' + file)
        total_time = data.time_signatures[1].time
        iteration = 0
        while iteration / note_divisions < total_time:
            notes.append(vocab_dict['0 0'])
            all_notes = [
                note for note in data.notes
                if is_note_valid(iteration / note_divisions, note, data.instrument_infos)
            ]
            all_notes.sort(key=lambda note: note.pitch, reverse=True)
            all_notes.sort(key=lambda note: data.instrument_infos[note.instrument].name)
            prev_instrument = None
            for note in all_notes:
                curr_instrument = data.instrument_infos[note.instrument].name
                if curr_instrument != prev_instrument:
                    if not prev_instrument or prev_instrument == 'p1':
                        if curr_instrument == 'tr':
                            notes.append(vocab_dict['2 0'])
                        if curr_instrument != 'p1':
                            notes.append(vocab_dict['2 0'])
                    else:
                        notes.append(vocab_dict['2 0'])
                    prev_instrument = curr_instrument
                if is_start(iteration / note_divisions, note.start_time):
                    notes.append(vocab_dict[str(1) + ' ' + str(note.pitch)])
                else:
                    notes.append(vocab_dict[str(3) + ' ' + str(note.pitch)])
            if not prev_instrument or prev_instrument == 'p1':
                notes.append(vocab_dict['2 0'])
                notes.append(vocab_dict['2 0'])
            elif prev_instrument == 'p2':
                notes.append(vocab_dict['2 0'])
            iteration += 1
    pickle.dump(notes, open('notes.p', 'wb'))
    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        network_input.append(notes[i:i + sequence_length])
        network_output.append(notes[i + sequence_length])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [4]:
def train_network():
    """ Train a Neural Network to generate music """
    vocab = get_vocab()
    notes = get_notes(vocab)
    n_vocab = len(vocab)
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    print('Done preparing sequences!')
    
    model = create_network(network_input, n_vocab)
    
    model.summary()
 
    checkpoint = ModelCheckpoint(
        "weights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]
    
    # Your line of code here
    model.fit(x=network_input, y=network_output, epochs=20, callbacks=callbacks_list)

In [5]:
train_network()

Done preparing sequences!
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 512)          528384    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 512)          1574912   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
activation (Activation)      (

In [6]:
def prepare_sequences_prediction(notes, n_vocab):
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        network_input.append(notes[i:i + sequence_length])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def generate_notes(model, network_input, vocab, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(vocab))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)

        index = np.random.choice(prediction.shape[1], p=(prediction[0] / sum(prediction[0])))
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [7]:
def generate():
    vocab = pickle.load(open('vocab.p', 'rb'))
    notes = pickle.load(open('notes.p', 'rb'))
    n_vocab = len(vocab)

    network_input, normalized_input = prepare_sequences_prediction(notes, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    
    model.load_weights('weights.hdf5')
    
    prediction_output = generate_notes(model, network_input, vocab, n_vocab)
    return prediction_output[200:300]

In [9]:
generate()

['2 0',
 '3 65',
 '2 0',
 '3 46',
 '2 0',
 '2 0',
 '2 0',
 '2 0',
 '2 0',
 '2 0',
 '2 0',
 '3 69',
 '1 48',
 '0 0',
 '3 70',
 '3 74',
 '3 72',
 '3 42',
 '3 43',
 '1 69',
 '3 43',
 '0 0',
 '1 77',
 '3 72',
 '3 67',
 '3 67',
 '0 0',
 '2 0',
 '3 65',
 '1 64',
 '2 0',
 '2 0',
 '3 67',
 '1 54',
 '0 0',
 '3 67',
 '3 67',
 '3 72',
 '3 48',
 '2 0',
 '3 42',
 '1 69',
 '3 43',
 '3 43',
 '3 72',
 '1 44',
 '0 0',
 '2 0',
 '3 65',
 '1 83',
 '2 0',
 '2 0',
 '0 0',
 '0 0',
 '2 0',
 '3 66',
 '1 67',
 '3 66',
 '3 55',
 '3 72',
 '3 58',
 '3 48',
 '3 67',
 '1 46',
 '3 43',
 '2 0',
 '3 70',
 '1 46',
 '0 0',
 '3 72',
 '3 70',
 '0 0',
 '2 0',
 '2 0',
 '2 0',
 '2 0',
 '3 43',
 '3 69',
 '1 74',
 '0 0',
 '3 72',
 '1 78',
 '3 46',
 '3 76',
 '3 40',
 '3 75',
 '1 72',
 '3 65',
 '0 0',
 '3 75',
 '3 40',
 '3 70',
 '1 58',
 '0 0',
 '2 0',
 '3 65',
 '3 72',
 '0 0',
 '0 0',
 '2 0']